In [1]:
print '123'
print 'hello world'

%load_ext autoreload
%autoreload 2

import pycuda.autoinit

123
hello world


In [6]:


"""
Multiplies two square matrices together using a *single* block of threads and
global memory only. Each thread computes one element of the resulting matrix.
"""

import numpy as np
from pycuda import driver, compiler, gpuarray, tools

# -- initialize the device
import pycuda.autoinit

kernel_code_template = """
#include "math.h"
#include <float.h>
#include "stdio.h"

__device__ void normalize_cc(float *a, float *b, float *c, int depth);

__device__ void normalize_dd(float *a, float *b, float *c, int depth)
{
    float len;
    float ta, tb, tc;
    float tt[3];
    
    ta = *a;
    tb = *b;
    tc = *c;
    len = ta*ta + tb*tb + tc*tc;
    
    if(len == 0.0)
        return;
    len = sqrt(len);
    *a = ta/len;
    *b = tb/len;
    *c = tc/len;
    
    if(depth <= 5)
        normalize_cc(a, b, c, depth+1);
        
}

__device__ void normalize_cc(float *a, float *b, float *c, int depth)
{
    float len;
    float ta, tb, tc;
    
    ta = *a;
    tb = *b;
    tc = *c;
    len = ta*ta + tb*tb + tc*tc;
    
    if(len == 0.0)
        return;
    len = sqrt(len);
    *a = ta/len;
    *b = tb/len;
    *c = tc/len;
    
    if(depth <= 5)
        normalize_dd(a, b, c, depth+1);
        
}

__global__ void MatrixMulKernel(float *a, float *b, float *c)
{
    // 2D Thread ID (assuming that only *one* block will be executed)
    int tx = threadIdx.x;
    int ty = threadIdx.y;

    // Pvalue is used to store the element of the matrix
    // that is computed by the thread
    float Pvalue = 0;

    // Each thread loads one row of M and one column of N,
    //   to produce one element of P.
    for (int k = 0; k < %(MATRIX_SIZE)s; ++k) {
        float Aelement = a[ty * %(MATRIX_SIZE)s + k];
        float Belement = b[k * %(MATRIX_SIZE)s + tx];
        Pvalue += Aelement * Belement;
    }
    
    float aa = 12.3;
    float bb = 10.0;
    
    normalize_cc(&Pvalue, &aa, &bb, 0);

    // Write the matrix to device memory;
    // each thread writes one element
    c[ty * %(MATRIX_SIZE)s + tx] = Pvalue;
}


"""

# define the (square) matrix size
#  note that we'll only use *one* block of threads here
#  as a consequence this number (squared) can't exceed max_threads,
#  see http://documen.tician.de/pycuda/util.html#pycuda.tools.DeviceData
#  for more information on how to get this number for your device
MATRIX_SIZE = 2

# create two random square matrices
a_cpu = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)
b_cpu = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)

# compute reference on the CPU to verify GPU computation
c_cpu = np.dot(a_cpu, b_cpu)

# transfer host (CPU) memory to device (GPU) memory
a_gpu = gpuarray.to_gpu(a_cpu)
b_gpu = gpuarray.to_gpu(b_cpu)

# create empty gpu array for the result (C = A * B)
c_gpu = gpuarray.empty((MATRIX_SIZE, MATRIX_SIZE), np.float32)

# get the kernel code from the template
# by specifying the constant MATRIX_SIZE

kernel_code = kernel_code_template.replace(
    '%(MATRIX_SIZE)s', str(MATRIX_SIZE)
    )

# compile the kernel code
mod = compiler.SourceModule(kernel_code)

# get the kernel function from the compiled module
matrixmul = mod.get_function("MatrixMulKernel")

# call the kernel on the card
matrixmul(
    # inputs
    a_gpu, b_gpu,
    # output
    c_gpu,
    # (only one) block of MATRIX_SIZE x MATRIX_SIZE threads
    block = (MATRIX_SIZE, MATRIX_SIZE, 1),
    )

# print the results
print "-" * 80
print "Matrix A (GPU):"
print a_gpu.get()

print "-" * 80
print "Matrix B (GPU):"
print b_gpu.get()

print "-" * 80
print "Matrix C (GPU):"
print c_gpu.get()

print "-" * 80
print "CPU-GPU difference:"
print c_cpu - c_gpu.get()

print np.allclose(c_cpu, c_gpu.get())


LaunchError: cuMemAlloc failed: unspecified launch failure

In [38]:
# raytracer.py - basic Python raytracer
# Micha Hanselmann, 2017
# ---
# based on http://www.scratchapixel.com/lessons/3d-basic-rendering/introduction-to-ray-tracing

from camera import Camera
from material import Material
from sphere import Sphere
from renderer import Renderer
from renderobject import RenderObject
from vector3 import Vector3

# render settings
width = 1280
height = 960
super_sampling = 1

# create demo scene
light_mat = Material(emission_color=Vector3(4, 4, 4))
light_sphere = Sphere(Vector3(5, 50, 20), 5)
light_obj = RenderObject(light_sphere, light_mat)

light2_mat = Material(emission_color=Vector3(2, 2, 2))
light2_sphere = Sphere(Vector3(0, 10, -5), 5)
light2_obj = RenderObject(light2_sphere, light2_mat)

blue_light_mat = Material(emission_color=Vector3(0, 0, 1))
blue_light_sphere = Sphere(Vector3(-40, 5, 10), 3)
blue_light_obj = RenderObject(blue_light_sphere, blue_light_mat)

ground_mat = Material(surface_color=Vector3(0.2, 0.2, 0.2))
ground_sphere = Sphere(Vector3(0, -10010, 20), 10000)
ground_obj = RenderObject(ground_sphere, ground_mat)

mat1 = Material(surface_color=Vector3(0.8, 0.2, 0.2))
sph1 = Sphere(Vector3(10, 1, 50), 5)
obj1 = RenderObject(sph1, mat1)

mat2 = Material(surface_color=Vector3(1, 1, 1), transparency=1, ior=1.1)
sph2 = Sphere(Vector3(1, -1, 10), 1)
obj2 = RenderObject(sph2, mat2)

mat3 = Material(surface_color=Vector3(0.8, 0.8, 1), reflectivity=0.5, transparency=0.8)
sph3 = Sphere(Vector3(-4, -1, 20), 2)
obj3 = RenderObject(sph3, mat3)

mat4 = Material(surface_color=Vector3(1, 1, 1), reflectivity=1.0)
sph4 = Sphere(Vector3(0, 0, 60), 8)
obj4 = RenderObject(sph4, mat4)

scene = [ground_obj, light_obj, light2_obj, blue_light_obj, obj1, obj2, obj3, obj4]
camera = Camera(Vector3(), 30)

In [3]:
print scene

[<renderobject.RenderObject instance at 0x0000000006A5B988>, <renderobject.RenderObject instance at 0x0000000006A5B588>, <renderobject.RenderObject instance at 0x0000000006A5B708>, <renderobject.RenderObject instance at 0x0000000006A5B848>, <renderobject.RenderObject instance at 0x0000000006A5BAC8>, <renderobject.RenderObject instance at 0x0000000006A5BC08>, <renderobject.RenderObject instance at 0x0000000006A5BD88>, <renderobject.RenderObject instance at 0x0000000006A5BEC8>]


In [4]:
camera = Camera(Vector3(), 30)

In [ ]:
renderer = Renderer(tilesize=64)


In [40]:
import numpy as np

aa = np.array([True, False, False], np.float32)
print aa[0]
ll = np.array([[]])
cc = np.array([[1,2,3],[1,2,3]])
ll = np.append(ll,cc[0])
ll = np.append(ll,cc[1])

print ll

1.0


ValueError: all the input arrays must have same number of dimensions

In [39]:
from tracer_gpu import Tracer_gpu
from tracer import Tracer
from ray import Ray

tracer = Tracer_gpu()
tracer_cpu = Tracer()

rendered_tile = {}
ray_array = []
ray_from_array = []

cpu_output = []
cpu_output0 = []
cpu_output1 = []
cpu_output2 = []

for y in range(0, height):
    for x in range(0, width):
        sum_color = Vector3()
        # sampled_rays = 0
        ray = camera.calcRay(x, y, width, height)
        
        #cpu_output.append(tracer_cpu.trace_diffuse(ray, scene))
#         out = tracer_cpu.trace_non_diffuse(ray, scene)
#         cpu_output0.append([out[0].origin, out[0].direction, out[0].current_ior])
#         cpu_output1.append([out[1].origin, out[1].direction, out[1].current_ior])
#         cpu_output2.append(out[2])
        ray_array.append([ray.origin.x, ray.origin.y, ray.origin.z, 
                          ray.direction.x, ray.direction.y, ray.direction.z, ray.current_ior])
        ray_from_array.append([x, y])
        # for ss_x in range(-super_sampling + 1, super_sampling):
        #     for ss_y in range(-super_sampling + 1, super_sampling):
        #         ray = camera.calcRay(x + ss_x, y + ss_y, width, height)
        #         sum_color += tracer.trace(ray, scene)
        #         sampled_rays += 1
        # rendered_tile[x, y] = sum_color * (1 / sampled_rays)

print tracer.trace
rendered_tile = tracer.trace(ray_array, ray_from_array, scene)
print rendered_tile.shape
print len(cpu_output)

<bound method Tracer_gpu.trace of <tracer_gpu.Tracer_gpu instance at 0x000000002E7ED208>>
(1228800L, 7L)
(8L, 16L)
(1228800L, 3L)
0


In [69]:
print rendered_tile[1][2]

print rendered_tile[2][2]

[array([0, 0, 2, 2, 4, 4, 5, 6, 6]), array([0.1014737 , 0.1014737 , 0.16294532, 0.16294532, 0.13196175,
       0.13196175, 0.16034472, 0.10850976, 0.10850976], dtype=float32), array([6, 6, 6, 6, 5, 5, 7, 5, 5]), array([[0.        , 0.        , 0.        ],
       [0.3288296 , 0.3288296 , 0.34647205],
       [0.        , 0.        , 0.        ],
       [0.33723915, 0.33723915, 0.3548629 ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.3193636 , 0.3193636 , 0.33704254]], dtype=float32)]
[[-4.85621    -1.4568331  21.74866    -0.21743694 -0.06522068  0.9738929
   1.        ]
 [-4.85621    -1.4568331  21.74866     0.6041674   0.37314838 -0.7040896
   1.        ]
 [-2.8948069  -1.447253   21.60565    -0.13258474 -0.06622455  0.9889568
   1.        ]
 [-2.8948069  -1.447253   21.60565    -0.94553906  0.26276436 -0.19212171
   1.        ]
 [ 0.5897499  -0.791089   10.8876095   0.10796723 -0.10429

In [52]:
cpu_output = []
cpu_output0 = []
cpu_output1 = []
cpu_output2 = []
cpu_out = []

for y in range(0, height):
    for x in range(0, width):
        sum_color = Vector3()
        # sampled_rays = 0
        ray = camera.calcRay(x, y, width, height)
        cpu_out.append(tracer_cpu.trace(ray, scene))
#         cpu_output.append(tracer_cpu.trace_diffuse(ray, scene))
        out = tracer_cpu.trace_non_diffuse(ray, scene)
        cpu_output0.append([out[0].origin, out[0].direction, out[0].current_ior])
        cpu_output0.append([out[1].origin, out[1].direction, out[1].current_ior])
        cpu_output2.append(out[2])

In [56]:
for ray in ray_ll:
    out = tracer_cpu.trace_non_diffuse(Ray(ray[0], ray[1]), scene)
    cpu_output1.append([out[0].origin, out[0].direction, out[0].current_ior])
    cpu_output1.append([out[1].origin, out[1].direction, out[1].current_ior])

In [57]:
diffuse_output = []
ray_ll = []
for ray in cpu_output1:
    if ray[-1] > 0:
        ray_ll.append(ray)
        diffuse_output.append(tracer_cpu.trace_diffuse(Ray(ray[0], ray[1]), scene))
print len(diffuse_output)        
print diffuse_output
print ray_ll

8
[Vector3(0, 0, 0), Vector3(0.328820195247, 0.328820195247, 0.346462302846), Vector3(0, 0, 0), Vector3(0.337234670053, 0.337234670053, 0.35485825488), Vector3(0, 0, 0), Vector3(0.675891392481, 0.675891392481, 0.675891392481), Vector3(0, 0, 0), Vector3(0.560598380727, 0.560598380727, 0.595062358285)]
[[Vector3(-4.85622996737, -1.45684899071, 21.7486449762), Vector3(0.604166870423, 0.373148230269, -0.704090044618), 1.0], [Vector3(-4.85631559465, -1.4568946779, 21.7488198494), Vector3(-0.217460105737, -0.0652380317212, 0.973886595877), 1.0], [Vector3(-2.89473300767, -1.44726650633, 21.6055957019), Vector3(-0.9455391729, 0.262764373101, -0.192121203255), 1.0], [Vector3(-2.89462247544, -1.44731123522, 21.6057562695), Vector3(-0.132496463429, -0.0662482317143, 0.988967066678), 1.0], [Vector3(0.589695271903, -0.791055061243, 10.8875765052), Vector3(0.773911920431, -0.443413475462, -0.452155757669), 1.0], [Vector3(0.58961320275, -0.791013268076, 10.8877540383), Vector3(0.170574285688, -0.1410

In [40]:
image = rendered_tile

def to_rgb_color(x, y, image):
        """Converts the vector into RGB values"""
        r = max(0, min(1, image[x+y*width, 0])) * 255
        g = max(0, min(1, image[x+y*width, 1])) * 255
        b = max(0, min(1, image[x+y*width, 2])) * 255
        return int(r), int(g), int(b)

file = open("output.ppm", "w")
file.write("P3\n{0} {1}\n255\n".format(width, height))
for y in range(height):
    for x in range(width):
        file.write("{0} {1} {2} ".format(*to_rgb_color(x,y,image)))
file.close()

In [76]:
image = cpu_output

file = open("output_cpu.ppm", "w")
file.write("P3\n{0} {1}\n255\n".format(width, height))
for y in range(height):
    for x in range(width):
        file.write("{0} {1} {2} ".format(*image[x+y*width].to_rgb_color()))
file.close()

In [15]:
import math
math.tan(math.pi*0.5*30/180)

0.2679491924311227